In [ ]:
# import libraries
import glob, os, openpyxl, re
import pandas as pd

In [ ]:
# Open Budget Folder to access all macro-enable Excel files
os.chdir("C:/Users/ElizaBui/Budget Templates/Line Item Budget Templates")

# Create an empty list to store Budget Template names
Budget_File_Names = []

# Loop through all macro-enable Excel files and push name to list
for file in glob.glob("*.xlsm"):
    Budget_File_Names.append(file)

In [ ]:
# Create an empty dictionary to store extracted data
    Data = {}

# Open each Budget File to loop thru each tab
for eachBudget in Budget_File_Names:
    # Keep track of progress
    print('Opening Workbook ' + eachBudget)
    wb = openpyxl.load_workbook(eachBudget, data_only = True)
    
    #Go through all tab names to check for those starting with 'Detail'
    TabNameList = []
    AllTabName = wb.get_sheet_names()
    for Tname in AllTabName:
        if Tname.startswith('Detail') and Tname.find('Master') == -1:
            TabNameList.append(Tname)
    
    # Loop through each 'Detail' tab to get Cost Center Name
    for tab in TabNameList:
        BudgetName = re.findall('^Budget.+-\s(.+)\s-.+.xlsm',eachBudget)[0]
        sheet = wb.get_sheet_by_name(tab)
        Cost_Center_Name = sheet['O' + str(5)].value
        
        # Loop through all rows to pull out data from rows starting with "LID" (Line Item Details) in col B
        for row in range(1, sheet.max_row + 1):
            if sheet['B' + str(row)].value == "LID":
                # extract row number
                R = int(row)
                
                # LIDs are NOT UNIQUE --> use Tab Name & Row No as UniqueID
                UniqueID = str(R) + Cost_Center_Name
                
                # Put row_number into Data dictionary
                Data[UniqueID]['Row_Number'] = R
                
                # Put business_group into Data dictionary
                Data[UniqueID]['Business_Group'] = BudgetName
                
                # Put cost center name into Data dictionary
                Data[UniqueID]['Cost_Center_Name'] = Cost_Center_Name
                
                # extract Line Item Item (LID) Name and put in Data dictionary
                Data[UniqueID]['LID_Name'] = sheet['P' + str(row)].value
                
                # extract Account Code
                Data[UniqueID]['Account_Code'] = str(sheet['C' + str(row)].value)
                
                # extract Grade
                Data[UniqueID]['Grade'] = Grade sheet['O' + str(row)].value
                
                # Process Contract Length and whether it's a Capex line item or not
                odd_value = sheet['Q' + str(row)].value
                CapEx = 0
                Contract_Length = 0
                if odd_value is not None and (type(odd_value) == float or type(odd_value) == int):
                    if odd_value < 1:
                        CapEx = odd_value
                    else:
                        Contract_Length = odd_value
                
                # extract CapEx identifier
                Data[UniqueID]['CapEx'] = CapEx
                
                # extract contract length
                Data[UniqueID]['Contract_Length'] = Contract_Length
                        
                # Extract monthly budgeted expense for each Line Item Detail (LID)
                Data[UniqueID]['Jan'] = sheet['AM' + str(row)].value
                Data[UniqueID]['Feb'] = sheet['AN' + str(row)].value
                Data[UniqueID]['Mar'] = sheet['AO' + str(row)].value
                Data[UniqueID]['Apr'] = sheet['AQ' + str(row)].value
                Data[UniqueID]['May'] = sheet['AR' + str(row)].value
                Data[UniqueID]['Jun'] = sheet['AS' + str(row)].value
                Data[UniqueID]['Jul'] = sheet['AU' + str(row)].value
                Data[UniqueID]['Aug'] = sheet['AV' + str(row)].value
                Data[UniqueID]['Sep'] = sheet['AW' + str(row)].value
                Data[UniqueID]['Oct'] = sheet['AY' + str(row)].value
                Data[UniqueID]['Nov'] = sheet['AZ' + str(row)].value
                Data[UniqueID]['Dec'] = sheet['BA' + str(row)].value
                Data[UniqueID]['Total Value'] = sheet['BC' + str(row)].value
                Data[UniqueID]['Comment'] = sheet['BE' + str(row)].value

# Convert the 'Data' dictionary into a DataFrame
df = pd.DataFrame.from_dict(Data, orient='index')

# Set 'UniqueID' as the index
df.index.name = 'UniqueID'

# Reset the index to have a default integer index
df.reset_index(inplace=True)
                
# Set the file path where you want to save the Excel file
output_excel_file = "Budget Line Item Details Consolidation.xlsx"

# Write the DataFrame to an Excel file
df.to_excel(output_excel_file, index=False)
print(f"Data has been written to {output_excel_file}")